In [ ]:
import os
import glob
from pathlib import Path
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Getting the dataset from kaggle

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = <...path to kaggle.json file>
%cd <...path to working directory>

In [ ]:
!kaggle datasets download -d crowww/a-large-scale-fish-dataset

In [ ]:
%ls

## Data extraction

In [ ]:
if not os.path.exists('Fish_Dataset'):
  %shell unzip a-large-scale-fish-dataset.zip -d ./
else:
  print('The dataset is already available')

In [ ]:
image_dir=Path('./Fish_Dataset/Fish_Dataset')
labels=list(image_dir.glob('**/*GT*/*.png'))

In [ ]:
images = list()

for folder in glob.glob('./Fish_Dataset/Fish_Dataset/*/**'):
  if not folder.endswith('GT'):
    path = folder+'/**'
    images.append(glob.glob(path))

images = [item for sublist in images for item in sublist]

In [ ]:
images = pd.Series(images, name='image').astype(str)
labels = pd.Series(labels,name='label').astype(str)
df = pd.concat((images, labels), axis=1)
df['fish_type'] = df['label'].apply(lambda x: x.split('/')[-3])

In [ ]:
df['fish_type'].value_counts()
# There 1000 images and 1000 labels for each class

In [ ]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img = mpimg.imread(images[0])
imgplot = plt.imshow(img)
plt.show()